- **Examples:** https://docs.manim.community/en/stable/examples.html


Few Summary From Code Snippet:
- Always extend from ``ThreeDScene`` for constructing 3d animations. It supports 3d camera movement.


## ThreeDSceneAttributes:
- ``phi`` => The polar angle i.e the angle between Z_AXIS and Camera **through ORIGIN** in **radians**. ( use * DEGREE for degrees)
- ``theta`` => The azimuthal angle i.e the **angle that spins the camera around the Z_AXIS** in **radians** (must use * DEGREE for degrees)



In [ ]:
# plane_explainer_combined.py
# Manim Community v0.19.0
#
# Run:
#   manim -pqh plane_explainer_combined.py PlaneFormsExplainer
#
# This unified scene first visualizes the Normal Form of a plane (Ax + By + Cz + D = 0)
# and then transitions to the Intercept Form (x/a + y/b + z/c = 1).
#
# Author: Surya Bhusal (refactored + combined by ChatGPT)

from manim import *
import numpy as np

config.background_color = BLACK


# ------------------------------------------------------------
# Utility math functions
# ------------------------------------------------------------
def normalize(vec):
    """Return a unit vector in the direction of `vec`."""
    arr = np.array(vec, dtype=float)
    n = np.linalg.norm(arr)
    return arr / n if n != 0 else arr


def plane_z_from_coeffs(A, B, C, D=0):
    """Given Ax + By + Cz + D = 0, return z = f(x,y)."""
    if C == 0:
        raise ValueError("C cannot be zero for z = f(x, y) form.")
    return lambda x, y: -(A * x + B * y + D) / C


# ------------------------------------------------------------
# Manim helper builders
# ------------------------------------------------------------
def make_plane_surface(axes, z_func, u_range, v_range, **style):
    """Create a parametric surface for plane z = f(x, y)."""
    def surf(u, v):
        x, y = u, v
        z = z_func(x, y)
        return axes.c2p(x, y, z)

    defaults = dict(resolution=(12, 12), fill_opacity=0.7)
    defaults.update(style)
    return Surface(surf, u_range=u_range, v_range=v_range, **defaults)


def make_vector(axes, direction, length=1.0, color=WHITE):
    """Arrow from origin in a given direction (scaled to `length`)."""
    d = normalize(direction)
    end = axes.c2p(*(d * length))
    return Arrow(start=ORIGIN, end=end, buff=0, color=color, tip_length=0.18)


def make_dot(axes, coords, color=WHITE):
    """Visible sphere (dot) at 3D coordinate."""
    return Sphere(radius=0.06).move_to(axes.c2p(*coords)).set_color(color)


def move_camera_along_normal(scene, normal, dist=1.8, run_time=2):
    """
    Move the camera smoothly along the plane's normal direction
    using Manim's built-in camera controls (compatible with v0.19.0).
    """
    nx, ny, nz = normalize(normal)

    # Determine small angular deltas based on direction
    d_theta = -np.arctan2(nx, ny) * 20 * DEGREES
    d_phi = -np.arctan2(nz, np.hypot(nx, ny)) * 10 * DEGREES

    # Use move_camera instead of camera.animate
    scene.move_camera(
        phi=scene.camera.phi + d_phi,
        theta=scene.camera.theta + d_theta,
        added_anims=[],
        run_time=run_time
    )


# ------------------------------------------------------------
# Unified animation scene
# ------------------------------------------------------------
class PlaneFormsExplainer(ThreeDScene):
    """
    Unified animation showing:
      1. Normal form: 2x + 3y + 5z = 0
      2. Transition to intercept form: x/a + y/b + z/c = 1
    """

    def construct(self):
        # === COMMON SETUP ===
        self.set_camera_orientation(phi=70 * DEGREES, theta=110 * DEGREES)
        axes = ThreeDAxes(
            x_range=[-5, 5, 1],
            y_range=[-5, 5, 1],
            z_range=[-5, 5, 1],
            axis_config={"color": GRAY_B},
        )
        self.add(axes)

        title = Tex("Plane Equation: Normal Form → Intercept Form", font_size=30)
        self.add_fixed_in_frame_mobjects(title)
        title.to_corner(UL).shift(LEFT * 1 + DOWN * 0.1)
        self.play(Write(title))
        self.wait(0.8)

        # === PART 1: NORMAL FORM ===
        A, B, C, D = 2, 3, 5, 0
        z_func = plane_z_from_coeffs(A, B, C, D)
        plane = make_plane_surface(axes, z_func, [-3, 3], [-3, 3], checkerboard_colors=[BLUE_D, BLUE_E])
        normal = np.array([A, B, C])
        normal_vec = make_vector(axes, normal, length=3, color=RED)
        in_plane = np.array([1, 1, -1])  # satisfies n·v = 0
        in_plane_vec = make_vector(axes, in_plane, length=2.5, color=YELLOW)

        eq1 = MathTex(r"2x + 3y + 5z = 0", font_size=32)
        eq2 = MathTex(r"\vec{n} = \langle 2,3,5 \rangle", color=RED, font_size=28)
        eq3 = MathTex(r"\vec{v} = \langle 1,1,-1 \rangle", color=YELLOW, font_size=28)
        for e, shift in zip([eq1, eq2, eq3], [UP * 2, UP * 1.3, UP * 0.6]):
            e.add_background_rectangle(opacity=0.2)
            self.add_fixed_in_frame_mobjects(e)
            e.to_edge(LEFT).shift(shift)

        self.play(Create(plane), run_time=1.4)
        self.play(GrowArrow(normal_vec), run_time=0.8)
        self.play(GrowArrow(in_plane_vec), run_time=0.8)
        self.wait(0.5)

        self.play(Write(eq1), Write(eq2), Write(eq3))
        self.wait(1)

        # Highlight perpendicularity visually
        square = Square(side_length=0.3, stroke_color=WHITE).move_to(axes.c2p(0.3, 0.1, 0.1))
        self.play(Create(square))
        self.wait(1.0)

        # Smooth camera motion along normal
        move_camera_along_normal(self, normalize(normal), dist=1.5)
        self.wait(0.8)

        # Transition text cue
        trans_text = Tex("Let's now relate it to the intercept form...", font_size=30)
        self.add_fixed_in_frame_mobjects(trans_text)
        trans_text.to_corner(DR).shift(UP * 0.5)
        self.play(FadeIn(trans_text))
        self.wait(1.3)
        self.play(FadeOut(VGroup(eq1, eq2, eq3, trans_text, plane, normal_vec, in_plane_vec, square)))

        # === PART 2: INTERCEPT FORM ===
        self.set_camera_orientation(phi=60 * DEGREES, theta=30 * DEGREES)
        self.wait(0.8)

        a, b, c = 4, 3, 5
        A_pt, B_pt, C_pt = (a, 0, 0), (0, b, 0), (0, 0, c)
        tri = Polygon(
            axes.c2p(*A_pt), axes.c2p(*B_pt), axes.c2p(*C_pt),
            fill_opacity=0.7, color=GREEN_C, stroke_width=2
        )
        dotA = make_dot(axes, A_pt, RED)
        dotB = make_dot(axes, B_pt, YELLOW)
        dotC = make_dot(axes, C_pt, BLUE)

        self.play(Create(tri), Create(dotA), Create(dotB), Create(dotC), run_time=1.6)

        # Labels
        labels = [
            MathTex(f"A=({a},0,0)", color=RED, font_size=24),
            MathTex(f"B=(0,{b},0)", color=YELLOW, font_size=24),
            MathTex(f"C=(0,0,{c})", color=BLUE, font_size=24),
        ]
        for lbl, (x, y, z) in zip(labels, [A_pt, B_pt, C_pt]):
            lbl.move_to(axes.c2p(x, y, z) + OUT * 0.3)
            self.add_fixed_orientation_mobjects(lbl)
            self.play(FadeIn(lbl), run_time=0.5)

        # Equation (fixed)
        eq_int = MathTex(r"\frac{x}{a} + \frac{y}{b} + \frac{z}{c} = 1", font_size=30)
        eq_ex = MathTex(
            rf"\frac{{x}}{{{a}}} + \frac{{y}}{{{b}}} + \frac{{z}}{{{c}}} = 1",
            font_size=32
        )
        for e, shift in zip([eq_int, eq_ex], [UP * 2, UP * 1.2]):
            e.add_background_rectangle(opacity=0.2)
            self.add_fixed_in_frame_mobjects(e)
            e.to_edge(LEFT).shift(shift)

        self.play(Write(eq_int))
        self.wait(0.5)
        self.play(Write(eq_ex))
        self.wait(1.0)

        self.play(Rotate(tri, angle=PI / 4, axis=OUT, run_time=2))
        self.wait(1)

        outro = Tex("Normal and Intercept forms describe the same plane geometrically!", font_size=30)
        self.add_fixed_in_frame_mobjects(outro)
        outro.to_edge(DOWN)
        self.play(Write(outro))
        self.wait(2)

%manim -ql -v warning PlaneFormsExplainer

In [ ]:
# plane_explainer_combined_fixed_latex.py
# Manim Community v0.19.0
#
# Unified scene: Normal form -> Intercept form
# - fixes: all LaTeX errors resolved
# - proper Tex/MathTex usage
# - labels, explanations, camera, vectors all properly displayed
#
from manim import *
import numpy as np

config.background_color = BLACK

# ---------------------------
# Math helpers
# ---------------------------
def normalize(vec):
    arr = np.array(vec, dtype=float)
    n = np.linalg.norm(arr)
    return arr / n if n != 0 else arr


def plane_z_from_coeffs(A, B, C, D=0):
    if C == 0:
        raise ValueError("C == 0: can't express as z = f(x,y)")
    return lambda x, y: -(A * x + B * y + D) / C


# ---------------------------
# Builders / small utilities
# ---------------------------
def make_plane_surface(axes, z_func, u_range, v_range, **style):
    def surf(u, v):
        x, y = u, v
        z = z_func(x, y)
        return axes.c2p(x, y, z)

    defaults = dict(resolution=(12, 12), fill_opacity=0.72)
    defaults.update(style)
    return Surface(surf, u_range=u_range, v_range=v_range, **defaults)


def make_vector(axes, direction, length=1.0, color=WHITE):
    d = normalize(direction)
    end = axes.c2p(*(d * length))
    return Arrow(start=ORIGIN, end=end, buff=0, color=color, tip_length=0.18)


def make_dot(axes, coords, color=WHITE):
    return Sphere(radius=0.06).move_to(axes.c2p(*coords)).set_color(color)


def move_camera_along_normal(scene, normal, run_time=2):
    nx, ny, nz = normalize(normal)
    d_theta = -np.arctan2(nx, ny) * 20 * DEGREES
    d_phi = -np.arctan2(nz, np.hypot(nx, ny)) * 10 * DEGREES
    scene.move_camera(
        phi=scene.camera.phi + d_phi,
        theta=scene.camera.theta + d_theta,
        run_time=run_time,
    )


# ---------------------------
# World-label updater
# ---------------------------
def attach_world_label_facing_camera(scene: ThreeDScene, label: Mobject, world_point_func, offset=OUT * 0.25):
    def updater(m):
        try:
            pt = np.array(world_point_func(), dtype=float)
            m.move_to(pt + offset)
        except Exception:
            pass
    label.add_updater(updater)
    return label


# ---------------------------
# Main unified scene
# ---------------------------
class PlaneFormsExplainer(ThreeDScene):
    def construct(self):
        # --- Setup axes and camera ---
        self.set_camera_orientation(phi=75 * DEGREES, theta=110 * DEGREES, distance=12)
        axes = ThreeDAxes(
            x_range=[-5, 5, 1],
            y_range=[-5, 5, 1],
            z_range=[-5, 5, 1],
            axis_config={"color": GRAY_B},
        )
        self.add(axes)

        # --- Title ---
        title = Tex("Plane: Normal form → Intercept form", font_size=28)
        title.to_corner(UR, buff=0.4)
        self.add_fixed_in_frame_mobjects(title)
        self.play(Write(title))
        self.wait(0.6)

        # --- Explanation panel ---
        explanation = VGroup()
        explanation.to_corner(UR).shift(DOWN * 0.9)
        self.add_fixed_in_frame_mobjects(explanation)

        def add_panel_text(mobj, wait=0.6, run=0.6):
            if len(explanation) == 0:
                mobj.to_corner(UR, buff=0.4)
            else:
                mobj.next_to(explanation[-1], DOWN, aligned_edge=LEFT, buff=0.16)
            explanation.add(mobj)
            self.play(Write(mobj), run_time=run)
            if wait:
                self.wait(wait)

        # --- PART 1: Normal form ---
        A, B, C, D = 2, 3, 5, 0
        z_fun = plane_z_from_coeffs(A, B, C, D)
        plane = make_plane_surface(axes, z_fun, u_range=[-3.2, 3.2], v_range=[-3.2, 3.2],
                                   checkerboard_colors=[BLUE_D, BLUE_E])
        plane.set_style(stroke_width=1.0)
        normal = np.array([A, B, C])
        normal_vec = make_vector(axes, normal, length=3.0, color=RED)
        in_plane_vec = make_vector(axes, [1, 1, -1], length=2.5, color=YELLOW)

        # Labels for arrows
        n_label = MathTex(r"\vec{n}", color=RED, font_size=26)
        v_label = MathTex(r"\vec{v}", color=YELLOW, font_size=26)
        attach_world_label_facing_camera(self, n_label, lambda: normal_vec.get_end(), offset=UP * 0.15 + RIGHT * 0.05)
        attach_world_label_facing_camera(self, v_label, lambda: in_plane_vec.get_end(), offset=DOWN * 0.12 + RIGHT * 0.03)

        self.add(plane, normal_vec, in_plane_vec, n_label, v_label)

        # Explanation panel (fixed) using Tex for mixed text + math
        add_panel_text(Tex(r"Normal form: $Ax + By + Cz + D = 0$", font_size=22))
        add_panel_text(Tex(r"Normal vector: $\vec{n} = \langle A, B, C\rangle$", font_size=20))
        add_panel_text(Tex(r"Example: $2x + 3y + 5z = 0 \Rightarrow \vec{n} = \langle 2,3,5\rangle$", font_size=20))
        add_panel_text(Tex(r"Pick $\vec{v}=\langle 1,1,-1\rangle$ then $\vec{n}\cdot\vec{v}=2+3-5=0$", font_size=20))

        self.play(Create(plane), run_time=1.4)
        self.play(GrowArrow(normal_vec), GrowArrow(in_plane_vec), run_time=0.9)
        self.wait(0.8)

        # Right angle marker
        right_angle = Square(side_length=0.28).set_fill(opacity=0).set_stroke(width=2).set_color(WHITE).scale(0.42)
        right_angle.move_to(axes.c2p(0.28, 0.12, 0.06))
        self.play(Create(right_angle), run_time=0.6)
        self.wait(0.6)

        move_camera_along_normal(self, normalize(normal), run_time=2.0)
        self.wait(0.8)

        self.play(FadeOut(VGroup(plane, normal_vec, in_plane_vec, n_label, v_label, right_angle)), run_time=0.8)
        self.wait(0.3)

        # --- PART 2: Intercept form ---
        self.set_camera_orientation(phi=60 * DEGREES, theta=30 * DEGREES, distance=12)
        self.wait(0.6)

        a, b, c = 4, 3, 5
        A_pt, B_pt, C_pt = (a, 0, 0), (0, b, 0), (0, 0, c)
        tri = Polygon(axes.c2p(*A_pt), axes.c2p(*B_pt), axes.c2p(*C_pt),
                      fill_opacity=0.75, color=GREEN_C, stroke_width=2)
        dotA = make_dot(axes, A_pt, RED)
        dotB = make_dot(axes, B_pt, YELLOW)
        dotC = make_dot(axes, C_pt, BLUE)

        self.play(Create(tri), Create(dotA), Create(dotB), Create(dotC), run_time=1.4)
        self.wait(0.4)

        # World-facing labels for intercepts
        labelA = MathTex(f"A=({a},0,0)", color=RED, font_size=22)
        labelB = MathTex(f"B=(0,{b},0)", color=YELLOW, font_size=22)
        labelC = MathTex(f"C=(0,0,{c})", color=BLUE, font_size=22)
        labelA.add_updater(lambda m: m.move_to(axes.c2p(*A_pt) + RIGHT * 0.25 + UP * 0.05))
        labelB.add_updater(lambda m: m.move_to(axes.c2p(*B_pt) + LEFT * 0.25 + UP * 0.03))
        labelC.add_updater(lambda m: m.move_to(axes.c2p(*C_pt) + OUT * 0.45 + RIGHT * 0.08 + UP * 0.02))
        self.add(labelA, labelB, labelC)
        self.play(FadeIn(labelA), FadeIn(labelB), FadeIn(labelC), run_time=0.6)
        self.wait(0.4)

        # Explanation panel for intercepts
        add_panel_text(Tex("Intercept form: points where plane meets axes", font_size=20))
        add_panel_text(Tex(r"$\frac{x}{a} + \frac{y}{b} + \frac{z}{c} = 1$", font_size=22))
        add_panel_text(Tex(r"Intercepts at $(a,0,0),(0,b,0),(0,0,c)$. Example: $a=4,b=3,c=5$", font_size=20))

        eq_example = MathTex(r"\frac{x}{" + str(a) + r"} + \frac{y}{" + str(b) + r"} + \frac{z}{" + str(c) + r"} = 1",
                             font_size=26)
        eq_example.set_color_by_tex(str(a), RED)
        eq_example.set_color_by_tex(str(b), YELLOW)
        eq_example.set_color_by_tex(str(c), BLUE)
        eq_example.next_to(explanation[-1], DOWN, aligned_edge=LEFT, buff=0.16)
        explanation.add(eq_example)
        self.play(Write(eq_example))
        self.wait(0.8)

        # Rotate triangle to show depth
        self.play(Rotate(tri, angle=PI / 8, axis=OUT, run_time=1.6))
        self.wait(0.6)

        # Outro
        outro = Tex("Both equations describe the same plane — different useful viewpoints.", font_size=22)
        outro.to_corner(DOWN)
        self.add_fixed_in_frame_mobjects(outro)
        self.play(Write(outro))
        self.wait(1.6)

        # Fade everything cleanly
        all_world = VGroup(tri, dotA, dotB, dotC, labelA, labelB, labelC)
        self.play(FadeOut(all_world), run_time=0.9)
        self.wait(0.3)
        self.play(FadeOut(explanation), run_time=0.6)
        self.wait(0.2)
        self.play(FadeOut(title), run_time=0.6)
        self.wait(0.2)


%manim -ql -v warning PlaneFormsExplainer